#imports



In [13]:
from google.colab import drive
drive.mount('/content/drive')
!pip install ipynb
import sys
sys.path.insert(0,'/content/drive/MyDrive/CNN-deterministic-decoupling')
path = '/content/drive/MyDrive/CNN-deterministic-decoupling'
import torch
import math
from ipynb.fs.defs.imageFunctions import prepare_image,show_image,save_image
from ipynb.fs.defs.Runge_Kutta import RungeKutta4
from ipynb.fs.defs.DISTS_mod import DISTS
from PIL import Image
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Transfer

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ref = prepare_image(Image.open(path+"/imagenes/texture4.jpg").convert("RGB"),size=128)
ref=ref.to(device)
ref.requires_grad_(False)


origin=prepare_image(Image.open(path+"/imagenes/texture6.jpg").convert("RGB"),size=128)
origin=origin.to(device)
origin.requires_grad_(True)

x0=origin.detach().clone()
x0=x0.to(device)


model_d = DISTS().to(device)
model_d.set_target(ref)

epoch=0
step_size = 0.01

inicial_Loss=model_d.forward(origin).item()
LossList=[inicial_Loss]
epoch+=1

while True:
    last_error=LossList[-1]
    RungeKutta4(origin,model_d.forward,step_size,LossList)
    epoch+=1
    if epoch%2000==0:
      print("iteracion "+str(epoch))
      print("loss "+str(LossList[-1]))
      show_image(origin)
      save_image(origin,path+'/Datos/RK4_t_6_4.png')
      with open(path+'/Datos/RK4_t_6_4.txt', 'w') as f:
        f.write(f'Total iterations: {epoch}\n')
        f.write('\n')
        f.write(f'Loss values: {LossList}\n')



save_image(origin,path+'/Datos/RK4_t_6_4.png')
show_image(origin)

In [ ]:
plt.plot(EpochList,ErrorList)
print((ErrorList[-1]/max(ErrorList))*100)
plt.grid(True)
print(ErrorList[-1])

In [ ]:
plt.plot(EpochList,LossList)
plt.grid(True)

In [ ]:
plt.plot(LossList,ErrorList)
plt.grid(True)